<a href="https://colab.research.google.com/github/DongUk-Park/RnD/blob/main/POI/NAIS/DataPreprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
import json
import csv
import os
import pandas as pd
import numpy as np
import random

#Data Untils

## RAW Data 생성

### Check_in Data 생성
리뷰 데이터로 체크인 데이터(user,business_id)를 만든다 <br>
원하는 데이터 정보 추가,삭제 : 13, 21 수정

In [66]:
# JSON 파일 경로
json_file_path = '/content/drive/MyDrive/dataset/yelp_dataset/review.json'

# CSV 파일 경로
csv_file_path = '/content/checkin.csv'

# CSV 파일을 쓰기 모드로 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    # CSV 라이터 생성
    csv_writer = csv.writer(csv_file)

    # CSV 파일 헤더 작성
    csv_writer.writerow(['user_id', 'business_id'])

    # JSON 파일을 한 줄씩 읽어서 처리
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            data = json.loads(line)  # JSON 데이터 파싱

            # user_id와 business_id 추출
            user_id = data['user_id']
            business_id = data['business_id']

            # CSV 파일에 데이터 작성
            csv_writer.writerow([user_id, business_id])

print("checkin_data 생성 완료.")


checkin_data 생성 완료.


In [67]:
"""
데이터 개수(row 수) 확인
"""

# CSV 파일 경로
csv_file_path = '/content/checkin.csv'

row_count = 0

# CSV 파일을 읽기 모드로 열고 행 수를 센다
with open(csv_file_path, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # 헤더 행을 건너뜁니다.
    for row in csv_reader:
        row_count += 1

print(f"CSV 파일의 행 수: {row_count}")

CSV 파일의 행 수: 6990280


### Business_info Data 생성
Business 데이터로 가게의 위치를 담은 데이터(business_id,location, city)를 만든다 <br>
원하는 데이터 정보 추가,삭제 : 13, 21 수정

In [68]:
# JSON 파일 경로
json_file_path = '/content/drive/MyDrive/dataset/yelp_dataset/business.json'

# CSV 파일 경로
csv_file_path = '/content/business_info.csv'

# CSV 파일을 쓰기 모드로 열기
with open(csv_file_path, 'w', newline='') as csv_file:
    # CSV 라이터 생성
    csv_writer = csv.writer(csv_file)

    # CSV 파일 헤더 작성
    csv_writer.writerow(['business_id_num','business_id','latitude ','longitute', 'city'])

    # business_id 를 int형으로 변환
    idx = 0

    # JSON 파일을 한 줄씩 읽어서 처리
    with open(json_file_path, 'r', encoding='utf-8') as json_file:
        for line in json_file:
            data = json.loads(line)  # JSON 데이터 파싱

            # user_id와 business_id 추출
            business_id_num = idx
            business_id = data['business_id']
            latitude = data['latitude']
            longitude = data['longitude']
            city = data['city']

            # CSV 파일에 데이터 작성
            csv_writer.writerow([business_id_num, business_id, latitude, longitude, city])
            idx += 1

print("CSV 파일 생성 완료.")


CSV 파일 생성 완료.


In [69]:
"""
데이터 개수(row 수) 확인
"""
# CSV 파일 경로
csv_file_path = '/content/business_info.csv'

row_count = 0

# CSV 파일을 읽기 모드로 열고 행 수를 센다
with open(csv_file_path, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    next(csv_reader)  # 헤더 행을 건너뜁니다.
    for row in csv_reader:
        row_count += 1

print(f"CSV 파일의 행 수: {row_count}")

CSV 파일의 행 수: 150346


In [70]:
"""
business_info에서 philadelphia에 속한 가게만 남김
"""

df = pd.read_csv('/content/business_info.csv')

# city 칼럼에서 "philadelphia"인 행만 선택
filtered_df = df[df['city'].str.contains("phila", case=False, na=False)]

# 필터링된 데이터를 CSV 파일로 저장
filtered_df.to_csv('/content/business_info_in_philadelphia.csv', index=False)

print("CSV 파일 필터링 및 저장 완료.")
filtered_df.shape

business_id_list = filtered_df['business_id_num'].tolist()
print(business_id_list[:5])

CSV 파일 필터링 및 저장 완료.
[3, 15, 19, 28, 31]


## 사용자 방문데이터(위치포함) 생성
위에서 생성한 데이터 두개를 합친 방문데이터 생성 <br>
checkin + business_info => location과 city 확인<br>
Philadelphia 도시에 대한 데이터만 남김

In [71]:
# business_info.csv 파일 경로
business_info_file = '/content/business_info.csv'

# checkin_data.csv 파일 경로
checkin_data_file = '/content/checkin.csv'

# 새로운 CSV 파일 경로 (결과 저장용)
output_file = '/content/checkin_data_in_philadelphia.csv'

# business_id를 키로 한 dictionary 생성
business_location = {}
with open(business_info_file, 'r', newline='') as business_file:
    csv_reader = csv.reader(business_file)
    next(csv_reader)  # 헤더 행 건너뛰기
    for row in csv_reader:
        business_id_num, business_id, latitude, longitude, city = row[0], row[1], row[2], row[3], row[4].lower() #city : 소문자로 받음
        business_location[business_id] = (business_id_num, latitude, longitude, city)

count = 0 # 잘 다 들어갔는지 체크
mcount = 0 # 잘 안들어간거 있는지 체크
not_phila = 0

# 새로운 CSV 파일(output_file : checkin_data_in_philadelphia)을 작성하여 business_id, latitude, longitude를 추가
with open(checkin_data_file, 'r', newline='') as checkin_file, open(output_file, 'w', newline='') as output_csv:
    csv_reader = csv.reader(checkin_file)
    csv_writer = csv.writer(output_csv)

    # 헤더 행 작성
    header = next(csv_reader)
    header.extend(['business_id', 'latitude', 'longitude', 'city'])
    csv_writer.writerow(header)

    for row in csv_reader:
        business_id = row[1]
        if business_id in business_location:
            business_id_num, latitude, longitude, city = business_location[business_id]

            # city가 "philadelphia"인 경우에만 데이터를 CSV 파일에 작성
            if "philadelphia" in city: #philadelphia 에서 오타가 존재할 수 있기 때문에 "phila"만 검사
              row.extend([business_id_num, latitude, longitude, city])
              # business_id 열을 제외한 열만 저장
              csv_writer.writerow(row[0:1] + row[2:])
              count += 1
            else:
              not_phila += 1
        else:
            # 해당 business_id가 business_info.csv에 없을 경우, 스킵
            mcount += 1
            continue

print("CSV 파일에 위치 정보가 추가되었습니다.")
print(count, mcount, not_phila) #philadelphia, 에러, 다른도시


CSV 파일에 위치 정보가 추가되었습니다.
968050 0 6022230


user_id 순으로 정렬

In [72]:
# CSV 파일 경로
input_file = '/content/checkin_data_in_philadelphia.csv'
output_file = '/content/checkin_data_final.csv'

# CSV 파일을 읽고 데이터를 리스트로 저장
data = []
with open(input_file, 'r', newline='') as csv_file:
    csv_reader = csv.reader(csv_file)
    header = next(csv_reader)  # 헤더 행
    data.append(header)
    for row in csv_reader:
        data.append(row)

# user_id를 기준으로 데이터를 정렬

print(data[:3])

data = data[1:] # header행 삭제
data.sort(key=lambda x: x[0])  # 여기서 0은 user_id 열을 가리킵니다. 0부터 시작하면 첫 번째 열입니다.

# user_id를 정수형으로 변환
idx = 0
before_user_id = data[0][0]
for i in data:
  if i[0] == before_user_id:
    i[0] = idx
  else:
    idx += 1
    before_user_id = i[0]
    i[0] = idx


# 정렬된 데이터를 새로운 파일에 저장
with open(output_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(data)

print("CSV 파일이 user_id를 기준으로 정렬되었고, 새로운 파일에 저장되었습니다.")

[['user_id', 'business_id', 'business_id', 'latitude', 'longitude', 'city'], ['OyoGAe7OKpv6SyGZT5g77Q', '12886', '39.9521029', '-75.1727526', 'philadelphia'], ['_7bHUi9Uuf5__HHc_Q8guQ', '6316', '40.0798480557', '-75.025079772', 'philadelphia']]
CSV 파일이 user_id를 기준으로 정렬되었고, 새로운 파일에 저장되었습니다.


In [73]:
# 데이터 확인
print(len(data))
data[13:30]

968050


[[4, '5115', '39.9559288', '-75.1574567', 'philadelphia'],
 [4, '14410', '39.9480501031', '-75.1469577849', 'philadelphia'],
 [4, '25084', '39.951903', '-75.161795', 'philadelphia'],
 [4, '25169', '39.9522798', '-75.1446095', 'philadelphia'],
 [4, '41292', '39.9495774962', '-75.1503095688', 'philadelphia'],
 [4, '41954', '39.9507587728', '-75.1438647821', 'philadelphia'],
 [4, '54520', '39.9486633342', '-75.1483974972', 'philadelphia'],
 [4, '57245', '39.949042', '-75.149198', 'philadelphia'],
 [4, '50952', '39.9488186002', '-75.1489040628', 'philadelphia'],
 [4, '51010', '40.0874923342', '-74.9616227288', 'philadelphia'],
 [4, '73288', '39.9488980012', '-75.1500296367', 'philadelphia'],
 [4, '66506', '39.9527511597', '-75.1424942017', 'philadelphia'],
 [4, '80270', '39.9509761191', '-75.1500126909', 'philadelphia'],
 [4, '89703', '39.9481737', '-75.1424642', 'philadelphia'],
 [4, '102262', '39.947326', '-75.147276', 'philadelphia'],
 [4, '130645', '39.9518574786', '-75.1461736562', 'p

In [74]:
"""
train에 사용될 history 데이터 (2차원 리스트)
인덱스 번호 : user_Id
인덱스 value(list형식) : 해당 uesr가 방문한 business_id 리스트
"""

history_list = []
tmp = []
before_user_id = 0
for idx, i in enumerate(data):
  if i[0] == before_user_id:
    tmp.append(i[1])
  else:
    if len(tmp) >= 10: # 방문 횟수가 10회가 넘는 유저만 append
      history_list.append(tmp)

    tmp = []
    tmp.append(i[1])
    before_user_id += 1

  if idx == len(data) - 1:
    history_list.append(tmp)

output_file = '/content/history_list.csv'
with open(output_file, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(history_list)

In [75]:
print(len(history_list))

sum = 0
for i in history_list:
  sum += len(i)
print(sum)

15919
465587


## Test,Validation,Train Data 생성

In [76]:
"""
위 history_list를 train, validation, test 데이터셋으로 분리시킴
"""

train_ratio = 0.6
test_ratio = 0.2
validation_ratio = 0.2

train_dataset = []
test_dataset = []
validation_dataset = []

train_negative_dataset = [] # business_id 개수 : 150346개
validation_negative_dataset = []
test_negative_dataset = []


#아래의 결과를 csv 파일로 저장할 경로, 총 6개의 파일 생성
output_train = '/content/history_train.csv'
output_validation = '/content/history_validation.csv'
output_test = '/content/history_test.csv'


for user, history in enumerate(history_list):
  ### 한 유저에 대해 train,validation,test dataset 생성
  total_samples = len(history) # 방문 횟수 저장
  random.shuffle(history) # 방문 기록 순서를 랜덤하게 섞어준다.

  train_size = int(total_samples * train_ratio)
  validation_size = int(total_samples * validation_ratio)

  list_test = history[(train_size + validation_size):]
  list_validation = history[train_size: (train_size + validation_size)]
  list_train = history[:train_size]


  train_dataset.append(list_train)
  validation_dataset.append(list_validation)
  test_dataset.append(list_test)

# output_train_negative = '/content/history_negative_train.csv'
# output_validation_negative = '/content/history_negative_validation.csv'
# output_test_negative = '/content/history_negative_test.csv'

# with open(output_train_negative, 'w', newline='') as csv_file1:
#   with open(output_validation_negative, 'w', newline='') as csv_file2:
#     with open(output_test_negative, 'w', newline='') as csv_file3:
#       csv_writer1 = csv.writer(csv_file1)
#       csv_writer2 = csv.writer(csv_file2)
#       csv_writer3 = csv.writer(csv_file3)

        # ### 한 유저에 대한 negative dataset 생성
        # neg_train = []
        # neg_val = []
        # neg_test = []
        # for i in range(14603): # business_id 개수만큼 반복
        #   if i in list_train:
        #     continue
        #   else:
        #     neg_train.append(i)
        #   if i in list_validation:
        #     continue
        #   else:
        #     neg_val.append(i)
        #   if i in list_test:
        #     continue
        #   else:
        #     neg_test.append(i)
        # csv_writer1.writerow(neg_train)
        # csv_writer2.writerow(neg_val)
        # csv_writer3.writerow(neg_test)

with open(output_train, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(train_dataset)
with open(output_validation, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(validation_dataset)
with open(output_test, 'w', newline='') as csv_file:
    csv_writer = csv.writer(csv_file)
    csv_writer.writerows(test_dataset)


In [82]:
print(len(train_dataset), train_dataset[:5])
print(len(validation_dataset), validation_dataset[:5])
print(len(test_dataset), test_dataset[:5])


# print(len(train_negative_dataset), train_negative_dataset[:5])
# print(len(validation_negative_dataset), validation_negative_dataset[:5])
# print(len(test_negative_dataset), test_negative_dataset[:5])

15919 [['125933', '66506', '89703', '50952', '14410', '80270', '25169', '25084', '102262', '73288'], ['76931', '125684', '46738', '12119', '54929', '87384', '137923', '43918', '93356', '10607'], ['67281', '28028', '22587', '5382', '124275', '78574', '55231', '71720'], ['116495', '49037', '74667', '123342', '59859', '149657', '93403'], ['45251', '22046', '61025', '141145', '105162', '141945']]
15919 [['130645', '57245', '5115'], ['131674', '54505', '35607'], ['19122', '41292'], ['144176', '76071'], ['94759', '29247']]
15919 [['51010', '41954', '54520', '41292'], ['52172', '69708', '52376', '43918', '132288'], ['56607', '110765', '110765', '8243'], ['98379', '44641', '116495'], ['115314', '138697', '36356']]


## 지역 임베딩 데이터 생성
NAIS 코드 완성 이후 진행 예정